# Activity : Extract and Repurpose Data (Part A)

Loading required libraries :

In [1]:
library("RSQLite")
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Initializing player and team tables and Getting an object to connect to database using dbConnect :

In [2]:
player <- NULL
team <- NULL
con <- dbConnect(drv=RSQLite::SQLite(), dbname="database.sqlite")

<h6>Q1) Which are the top 10 players by overall rating? </h6>

Loading Player_Attributes table from database :

In [3]:
p1 <-  NULL
p1 = dbGetQuery( con,'select * from Player_Attributes' )

Extracting required columns from table and converting to a tbl :

In [4]:
p1tbl <- tbl_df(data.frame(api_id=p1$player_api_id, OverallRating=p1$overall_rating))

1) Grouping table by player_api_id <br>
2) Summarising ny taking max of overall_rating over each group in table <br>
3) Arranging table in descending order of overall_rating <br>
4) Extracting top 10 records using head. 

In [5]:
#1)
p1tbl <- group_by(p1tbl, api_id)
#2)
p1tbl <- summarise(p1tbl, OverallRating=max(OverallRating))
#3)
p1tbl <- arrange(p1tbl, desc(OverallRating))
#4)
p1tbl <- head(p1tbl, 10)

Retrieving player_names from table using select query

In [6]:
top10overall<- NULL
for(id in p1tbl$api_id){
    p2 =dbGetQuery(con, paste("select player_name from Player where player_api_id = ",id,""))
    top10overall = rbind(top10overall, p2)
}
top10overall <- cbind(top10overall,OverallRating=p1tbl$OverallRating)
colnames(top10overall) <- c("PlayerNamesByOverall","ByOverallRating")

Binding results to the player table

In [7]:
player <- bind_cols(player, top10overall)

<h6>Q2) Which are the top 10 goalkeepers by sum of gk attributes?</h6>

1) Mutating sum of gk attribute rows to a new table <br>
2) Converting to a tbl. <br>
3) Grouping table by player_api_id <br>
4) Summarising by taking max of gk attributes over each group in table <br>
5) Arranging table in descending order of gk attributes. <br>
6) Extracting top 10 records using head. 

In [8]:
#1)
p2tbl <- mutate(p1,top10gk = rowSums(p1[, c(38,39,40,41,42)]))
#2)
p2tbl <- tbl_df(data.frame(api_id=p1$player_api_id, GKRating=p2tbl$top10gk))
#3)
p2tbl <- group_by(p2tbl, api_id)
#4)
p2tbl <- summarise(p2tbl, GKRating=max(GKRating))
#5)
p2tbl <- arrange(p2tbl, desc(GKRating))
#6)
p2tbl <- head(p2tbl, 10)

Retrieving player_names from table using select query

In [9]:
top10GKR<- NULL
for(id in p2tbl$api_id){
    p3 =dbGetQuery(con, paste("select player_name from Player where player_api_id = ",id,""))
    top10GKR = rbind(top10GKR, p3)
}
top10GKR <- cbind(top10GKR,OverallRating=p2tbl$GKRating)
colnames(top10GKR) <- c("PlayerNamesByGKR","ByGKRating")

Binding results to the player table

In [10]:
player <- bind_cols(player, top10GKR)

<h6>Q3) Which are the top 10 players by number of appearances (with any team they played with)?</h6>

Loading match table from database :

In [11]:
p4 <-  NULL
p4 = dbGetQuery(con,'select * from match')

Binding home_players and away_players in one column using rbind. <br>
Grouping table by player_api_id <br>
Summarising by taking a sum of th number of times a player has appeared in the column. <br>
Arranging table in descending order of no. of appearances.

In [12]:
full <- tbl_df(na.omit(p4[,56]))
full <- count(full, value)
for(x in seq(57,77)) {
    h1 <- tbl_df(na.omit(p4[,x]))
    h1_val <- count(h1, value)
    full <- bind_rows(h1_val,full)
    full_group <- summarise(group_by(full,value),count=sum(n))
    full_group <- arrange(full_group, desc(count))
}  

Extracting top ten appeared players using head. <br>
Retrieving player_names from table using select query <br>

In [13]:
top10byappear <- head(full_group, 10)
appearances <- NULL
for(id in top10byappear$value){
    p7 =dbGetQuery(con, paste("select player_name from player where player_api_id = ",id,""))
    appearances = rbind(appearances, p7)
}
appearances <- cbind(appearances, top10byappear$count)
colnames(appearances) <- c("PlayerNamesByAppear","ByAppearances")

Binding results to the player table :

In [14]:
player <- bind_cols(player, appearances)

<h6>Q4) Which are the top 10 players by number of leagues they played in?</h6>

Binding home_players and away_players along with league ids in one table using bind_rows. <br>

In [15]:
df <- data.frame(id=p4[,56], league=p4[,3])
full <- tbl_df(na.omit(df))

In [16]:
for(x in seq(57,77)) {
    df <- data.frame(id=p4[,x], league=p4[,3])
    h1 <- tbl_df(na.omit(df))
    #h1_val <- count(h1, value)
    full <- bind_rows(h1,full)
}

Grouping table by player_api_id <br>
Summarising by taking length of the unique elements in league column in a group. <br>
Arranging table in descending order of no. of leagues.

In [17]:
groupbyid <- group_by(full,id)
full_group <- summarise(groupbyid, leagues=length(unique(league)))
full_group <- arrange(full_group, desc(leagues))  

Extracting top ten appeared players using head. <br>
Retrieving player_names from table using select query <br>

In [18]:
top10byleagues <- head(full_group, 10)
leagues <- NULL
for(id in top10byleagues$id){
    p8 =dbGetQuery(con, paste("select player_name from player where player_api_id = ",id,""))
    leagues = rbind(leagues, p8)
}
leagues <- cbind(leagues, top10byleagues$leagues)
colnames(leagues) <- c("PlayerNamesByLeages","ByLeagues")

Binding results to the player table

In [19]:
player <- bind_cols(player, leagues)

<h6>Q5) Which are the top 10 teams by sum of build up play attributes?</h6>

Loading team_attributes table from database :

In [20]:
p5 <-  NULL
p5 = dbGetQuery( con,'select * from team_attributes' )

1) Mutating sum of buld up play attribute rows to a new table <br>
2) Converting to a tbl. <br>
3) Grouping table by team_api_id <br>
4) Summarising by taking max of build attributes over each group in table <br>
5) Arranging table in descending order of build attributes. <br>
6) Extracting top 10 records using head. 

In [21]:
#1)
p5 <- mutate(p5,top10build = rowSums(p5[, c(5,7,9)]))
#2)
p5tbl <- tbl_df(na.omit(data.frame(api_id=p5$team_api_id, BuildAttr = p5$top10build)))
#3)
p5tbl <- group_by(p5tbl, api_id)
#4)
p5tbl <- summarise(p5tbl, BuildAttr = max(BuildAttr))
#5)
p5tbl <- arrange(p5tbl, desc(BuildAttr))
#6)
p5tbl <- head(p5tbl,10)

Retrieving team_names from table using select query

In [22]:
build_team_name <- NULL
for(id in p5tbl$api_id){
    p6 =dbGetQuery(con, paste("select team_long_name from team where team_api_id = ",id,""))
    build_team_name = rbind(build_team_name, p6)
}
build_team_name <- cbind(build_team_name, p5tbl$BuildAttr)
colnames(build_team_name) <- c("TeamNamesByBuild","ByBuild")

Binding results to the team table :

In [23]:
team <- bind_cols(team, build_team_name)

<h6>Q6) Which are the top 10 teams by sum of chance creation attributes?</h6>

1) Mutating sum of chance creation attribute rows to a new table <br>
2) Converting to a tbl. <br>
3) Grouping table by team_api_id <br>
4) Summarising by taking max of chance attributes over each group in table <br>
5) Arranging table in descending order of chance attributes. <br>
6) Extracting top 10 records using head. 

In [24]:
#1)
p6tbl <- mutate(p5,top10chance = rowSums(p5[, c(12,14,16)]))
#2)
p6tbl <- tbl_df(data.frame(api_id=p6tbl$team_api_id, ChanceAttr=p6tbl$top10chance))
#3)
p6tbl <- group_by(p6tbl, api_id)
#4)
p6tbl <- summarise(p6tbl, ChanceAttr=max(ChanceAttr))
#5)
p6tbl <- arrange(p6tbl, desc(ChanceAttr))
#6)
p6tbl <- head(p6tbl, 10)

Retrieving team_names from table using select query :

In [25]:
chance_team_name <- NULL
for(id in p6tbl$api_id){
    p6 =dbGetQuery(con, paste("select team_long_name from team where team_api_id = ",id,""))
    chance_team_name = rbind(chance_team_name, p6)
}
chance_team_name <- cbind(chance_team_name, p6tbl$ChanceAttr)
colnames(chance_team_name) <- c("TeamNamesByChance","ByChanceAttributes")

Binding results to the team table :

In [26]:
team <- bind_cols(team, chance_team_name)

<h6>Q7) Which are the top 10 teams by sum of defense attributes?</h6>

1) Mutating sum of defense attribute rows to a new table <br>
2) Converting to a tbl. <br>
3) Grouping table by team_api_id <br>
4) Summarising by taking max of defense attributes over each group in table <br>
5) Arranging table in descending order of defense attributes. <br>
6) Extracting top 10 records using head. 

In [27]:
#1)
p7tbl <- mutate(p5,top10def = rowSums(p5[, c(19,21,23)]))
#2)
p7tbl <- tbl_df(data.frame(api_id=p7tbl$team_api_id, DefAttr=p7tbl$top10def))
#3)
p7tbl <- group_by(p7tbl, api_id)
#4)
p7tbl <- summarise(p7tbl, DefAttr=max(DefAttr))
#5)
p7tbl <- arrange(p7tbl, desc(DefAttr))
#6)
p7tbl <- head(p7tbl, 10)

Retrieving team_names from table using select query :

In [28]:
def_team_name <- NULL
for(id in p7tbl$api_id){
    p7 =dbGetQuery(con, paste("select team_long_name from team where team_api_id = ",id,""))
    def_team_name = rbind(def_team_name, p7)
}
def_team_name <- cbind(def_team_name, p7tbl$DefAttr)
colnames(def_team_name) <- c("TeamNamesByDef","byDefAtrributes")

Binding results to the team table :

In [29]:
team <- bind_cols(team, def_team_name)

<h6>Q8) Which are the top 10 teams by number of scored goals?</h6>

1) Extracting home_team and goals columns from table and converting to table. <br>
2) Extracting away_team and goals columns from table and converting to table. <br>
3) Binding both into 1 tbl using bind_rows.<br>
4) Grouping by team_id and summarising by taking a sum over goals of each team group. <br>
5) Arranging table in descending order of no. of goals. <br>
6) Extracting top 10 records using head.

In [30]:
#1)
df1 <- data.frame(team_id=p4[,8], goals=p4[,10])
tbl1 <- tbl_df(na.omit(df1))
#2)
df2 <- data.frame(team_id=p4[,9], goals=p4[,11])
tbl2 <- tbl_df(na.omit(df2))
#3)
tbl1 <- bind_rows(tbl1, tbl2)
#4)
goals <- summarise(group_by(tbl1, team_id), goals=sum(goals))
#5)
goals <- arrange(goals, desc(goals))  
#6)
top10bygoals <- head(goals, 10)

Retrieving team_names from table using select query :

In [31]:
goal <- NULL
for(id in top10bygoals$team_id){
    p9 =dbGetQuery(con, paste("select team_long_name from team where team_api_id = ",id,""))
    goal = rbind(goal, p9)
}
goal <- cbind(goal, top10bygoals$goals)
colnames(goal) <- c("TeamsByGoals","ByGoals")

Binding results to the player table :

In [32]:
team <- bind_cols(team, goal)

Writng tables to respective CSV.

In [33]:
write.csv(team, "team.csv")
write.csv(player, "player.csv")